# Pose Calculation of Robot with Aruco markers and Computer Vision using ChatGPT

### Prompt into ChatGPT

## Import libraries

In [1]:
import cv2
import numpy as np
import math

## Define Functions

In [2]:
def find_aruco_markers(img):
    # Load the dictionary that was used to generate the markers.
    aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_6X6_250)
    # Initialize the detector parameters using default values
    parameters = cv2.aruco.DetectorParameters_create()
    # Detect the markers in the image
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(img, aruco_dict, parameters=parameters)
    return corners, ids

def calculate_robot_position(corners, ids):
    # Extract corners of the two bounding box markers
    top_left_marker = corners[np.where(ids == 1)[0][0]]
    bottom_right_marker = corners[np.where(ids == 2)[0][0]]
    robot_marker = corners[np.where(ids == 3)[0][0]]

    # Calculate center points
    top_left_center = np.mean(top_left_marker[0], axis=0)
    bottom_right_center = np.mean(bottom_right_marker[0], axis=0)
    robot_center = np.mean(robot_marker[0], axis=0)

    # Calculate position of the robot relative to top left marker
    x = robot_center[0] - top_left_center[0]
    y = robot_center[1] - top_left_center[1]

    # Calculate angle of the robot marker
    robot_vector = robot_marker[0][1] - robot_marker[0][0]  # vector from corner 0 to corner 1
    theta = math.atan2(robot_vector[1], robot_vector[0])

    return (top_left_center, bottom_right_center, (x, y, theta))

def locate_robot(img):
    corners, ids = find_aruco_markers(img)
    if ids is None or len(ids) < 3:
        raise ValueError("Not all required markers were found in the image.")
    return calculate_robot_position(corners, ids)

In [3]:
# Example usage
if __name__ == "__main__":
    img = cv2.imread('image_with_aruco_markers.jpg')  # Load your image here
    top_left, bottom_right, robot_position = locate_robot(img)
    print("Top Left Marker Position:", top_left)
    print("Bottom Right Marker Position:", bottom_right)
    print("Robot Position (x, y, theta):", robot_position)


AttributeError: module 'cv2.aruco' has no attribute 'Dictionary_get'